# Creación de mapas

In [41]:
import os
import pandas as pd
import numpy as np
import json
import random
import folium
import branca.colormap as cm

## Generar simulación

- Se genera un número random
- Con la matriz estocástica se saca el estado por cada país
- Se intera n veces, dependiendo de cuántos años se quieran simular

In [42]:
markov_matrices = None

with open(os.path.join("Clean","markov.json")) as f:
    markov_matrices = json.load(f)

In [43]:
def final_state(country, n):
    if country not in markov_matrices:
        return 5
    matrix = markov_matrices[country]["Matrix"]
    initialState = 0
    for i in range(len(matrix)):
        if matrix[i][i]>initialState:
            initialState = i
    for i in range(n):
        prob = random.random()
        for j in range(len(matrix[initialState])):
            prob -= matrix[initialState][j]
            if prob <= 0:
                initialState = j
                break
    finalState = markov_matrices[country]["States"][initialState]
    return finalState

In [44]:
print(final_state("Chile", 100))

1


## Clasificación de colores
### Para representar las clasificaciones de temperaturas se usan colores:
- <b>Muy frío:</b> Azul
- <b>Frío:</b> Amarillo
- <b>Templado:</b> Verde
- <b>Caliente:</b> Naranja
- <b>Muy caliente:</b> Rojo
- <b>Desconocido:</b> Blanco

In [60]:
step = cm.StepColormap(['blue','yellow','green','orange','red', 'white'], vmin=0, vmax=5, index=[0,0.9,1.9,2.9,3.9,4.9], caption='step')

In [62]:
m1 = folium.Map(location=[0, 0], zoom_start=1)
geo = {"type":"FeatureCollection","features":[]}

path = 'GeoJSON'
for i in os.listdir(path):
    f = os.path.join(path, i)
    with open(f) as file:
        data = json.load(file)
        geo["features"].append(data["features"][0])
        
folium.GeoJson(geo,style_function=lambda feature: {
        'fillColor': step(final_state(feature["properties"]["name"], 100)),
        'color' : 'black',
        'weight' : 2,
        'fillOpacity': 0.8,
        'dashArray' : '5, 5'
        }).add_to(m1)
m1

## Coordenadas

- Los GeoJSON con las coordenadas para para dibujar el mapa fueron sacadas del repositorio [world.geo.json](https://github.com/johan/world.geo.json/tree/master/countries)